In [14]:
import cosmofast as cf
from astropy.io import fits
import numpy as np

In [15]:
from scipy.io import FortranFile
cov_low = FortranFile(
    'cosmofast/planck_18/data/legacy-plik_lite/c_matrix_plik_v22.dat').read_reals().reshape((613, 613))
cov = cov_low + cov_low.T - np.diag(np.diag(cov_low))

mu = np.loadtxt('cosmofast/planck_18/data/legacy-plik_lite/cl_cmb_plik_v22.dat')[:, 1].copy()
blmin = np.loadtxt('cosmofast/planck_18/data/legacy-plik_lite/blmin.dat')
blmax = np.loadtxt('cosmofast/planck_18/data/legacy-plik_lite/blmax.dat')
bweight = np.loadtxt('cosmofast/planck_18/data/legacy-plik_lite/bweight.dat')

weight_t = np.copy(bweight[int(blmin[0]):int(blmax[214] + 1)])
weight_e = np.copy(bweight[int(blmin[0]):int(blmax[198] + 1)])

low_t = np.copy(blmin[:215]).astype(np.int32)
width_t = np.copy((blmax - blmin + 1)[:215]).astype(np.int32)
low_e = np.copy(blmin[:199]).astype(np.int32)
width_e = np.copy((blmax - blmin + 1)[:199]).astype(np.int32)

cov_inv_ttteee = np.linalg.inv(cov)
cov_inv_ttteee_low = np.linalg.cholesky(cov_inv_ttteee)
mu_ttteee = mu @ cov_inv_ttteee_low

In [3]:
test = np.load("cosmofast/tests/test_plik_lite_ttteee.npz")
test['test_param'].shape

(7528,)

In [4]:
cls_b = np.empty(613)

# power[30:, 0]
cf.planck_18._plik_lite._get_binned_cls(
    test['test_param'][30:2509], cls_b[:215], weight_t, low_t, width_t, 215)

# power[30:, 3]
cf.planck_18._plik_lite._get_binned_cls(
    test['test_param'][(2509+2509+30):(2509+2509+2509)], cls_b[215:414], weight_e, low_e, width_e, 199)

# power[30:, 1]
cf.planck_18._plik_lite._get_binned_cls(
    test['test_param'][(2509+30):(2509+2509)], cls_b[414:613], weight_e, low_e, width_e, 199)

cls_b = cls_b @ cov_inv_ttteee_low

In [5]:
test_ttteee = np.empty(1)
cf.planck_18._plik_lite._plik_lite_f(
    cls_b, test['test_param'][-1], test_ttteee, mu_ttteee, 613)
test_ttteee, test['test_value']

(array([-292.16906205]), array([-292.16906205]))

In [6]:
cov_inv_tt = np.linalg.inv(cov[:215, :215])
cov_inv_tt_low = np.linalg.cholesky(cov_inv_tt)
mu_tt = mu[:215] @ cov_inv_tt_low

In [7]:
test = np.load("cosmofast/tests/test_plik_lite_tt.npz")
test['test_param'].shape

(2510,)

In [8]:
cls_b = np.empty(215)

# power[30:, 0]
cf.planck_18._plik_lite._get_binned_cls(
    test['test_param'][30:2509], cls_b, weight_t, low_t, width_t, 215)

cls_b = cls_b @ cov_inv_tt_low

In [9]:
test_tt = np.empty(1)
cf.planck_18._plik_lite._plik_lite_f(
    cls_b, test['test_param'][-1], test_tt, mu_tt, 215)
test_tt, test['test_value']

(array([-103.46972499]), array([-103.46972499]))

In [10]:
np.savez_compressed(
    'cosmofast/planck_18/data/plik_lite_tt.npz', weight_t=weight_t, low_t=low_t,
    width_t=width_t, n_bin_t=215, cov_inv_low=cov_inv_tt_low, mu=mu_tt,
    n_bin=215)
np.savez_compressed(
    'cosmofast/planck_18/data/plik_lite_ttteee.npz', weight_t=weight_t,
    low_t=low_t, width_t=width_t, n_bin_t=215, weight_e=weight_e, low_e=low_e,
    width_e=width_e, n_bin_e=199, cov_inv_low=cov_inv_ttteee_low, mu=mu_ttteee,
    n_bin=613, i_bin=[0, 215, 414, 613])

In [11]:
import cosmofast as cf
import numpy as np
f_tt = np.load('cosmofast/planck_18/data/plik_lite_tt.npz')
f_ttteee = np.load('cosmofast/planck_18/data/plik_lite_ttteee.npz')
test_tt = np.load("cosmofast/tests/test_plik_lite_tt.npz")
test_ttteee = np.load("cosmofast/tests/test_plik_lite_ttteee.npz")

In [12]:
cls_b = np.empty(f_ttteee['n_bin'])

# power[30:, 0], TT
cf.planck_18._plik_lite._get_binned_cls(
    test_ttteee['test_param'][30:2509],
    cls_b[f_ttteee['i_bin'][0]:f_ttteee['i_bin'][1]], f_ttteee['weight_t'],
    f_ttteee['low_t'], f_ttteee['width_t'], f_ttteee['n_bin_t'])

# power[30:, 3], TE
cf.planck_18._plik_lite._get_binned_cls(
    test_ttteee['test_param'][(2509+2509+30):(2509+2509+2509)],
    cls_b[f_ttteee['i_bin'][1]:f_ttteee['i_bin'][2]], f_ttteee['weight_e'],
    f_ttteee['low_e'], f_ttteee['width_e'], f_ttteee['n_bin_e'])

# power[30:, 1], EE
cf.planck_18._plik_lite._get_binned_cls(
    test_ttteee['test_param'][(2509+30):(2509+2509)],
    cls_b[f_ttteee['i_bin'][2]:f_ttteee['i_bin'][3]], f_ttteee['weight_e'],
    f_ttteee['low_e'], f_ttteee['width_e'], f_ttteee['n_bin_e'])

cls_b = cls_b @ f_ttteee['cov_inv_low']

ttteee = np.empty(1)
cf.planck_18._plik_lite._plik_lite_f(
    cls_b, test_ttteee['test_param'][-1], ttteee, f_ttteee['mu'],
    f_ttteee['n_bin'])
ttteee, test_ttteee['test_value']

(array([-292.16906205]), array([-292.16906205]))

In [13]:
cls_b = np.empty(f_tt['n_bin'])

# power[30:, 0]
cf.planck_18._plik_lite._get_binned_cls(
    test_tt['test_param'][30:2509], cls_b, f_tt['weight_t'], f_tt['low_t'],
    f_tt['width_t'], f_tt['n_bin_t'])

cls_b = cls_b @ f_tt['cov_inv_low']

tt = np.empty(1)
cf.planck_18._plik_lite._plik_lite_f(
    cls_b, test_tt['test_param'][-1], tt, f_tt['mu'], f_tt['n_bin'])
tt, test_tt['test_value']

(array([-103.46972499]), array([-103.46972499]))